In [21]:
#importing dependencies
import pandas as pd
import os

# set the path for the files
school_data_to_load = os.path.join('Resources','schools_complete.csv')
student_data_to_load = os.path.join('Resources', 'students_complete.csv')

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [33]:
## DISTRICT SUMMARY ##

#getting the values for the first data frame
total_schools = school_data_complete['school_name'].nunique()
total_students = school_data_complete['student_name'].count()
total_budget = school_data['budget'].sum()
avg_math_score = round(school_data_complete ['math_score'].mean(),2)
avg_reading_score = round(school_data_complete['reading_score'].mean(),2)

#find the kids that are passing
#math
math_passing = school_data_complete[school_data_complete['math_score'] >= 70]
math_passing_counts = math_passing['math_score'].count()
#reading
reading_passing = school_data_complete[school_data_complete['reading_score'] >= 70]
reading_passing_counts = reading_passing['reading_score'].count()
#both
both_passing = school_data_complete[(school_data_complete['math_score'] >= 70) & 
                                    (school_data_complete['reading_score'] >= 70)]
both_passing_counts = both_passing['student_name'].count()

#calculations for the percentages
passing_perentage_math = round((int(math_passing_counts)/total_students)*100,2)
passing_percentage_reading = round((int(reading_passing_counts)/total_students)*100,2)
passing_percentage_both = round((int(both_passing_counts)/total_students)*100,2)

#creating the data frame
#dictionaries
summary_dictionaries = [{'Total Schools': total_schools, 'Total Students': total_students, 
                        'Total Budget': total_budget, 'Avg Math Score': avg_math_score,
                       'Avg Reading Score': avg_reading_score, '% Passing Math': passing_perentage_math,
                        '% Passing Reading': passing_percentage_reading, '% Passing Overall': passing_percentage_both}]

#makin the frame
district_summary_df = pd.DataFrame(summary_dictionaries)

#formatting
district_summary_df['Total Students'] = district_summary_df['Total Students'].map('{:,}'.format)
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].map('${:,}'.format)
district_summary_df['% Passing Math'] = district_summary_df['% Passing Math'].map('{:.2f}%'.format)
district_summary_df['% Passing Reading'] = district_summary_df['% Passing Reading'].map('{:.2f}%'.format)
district_summary_df['% Passing Overall'] = district_summary_df['% Passing Overall'].map('{:.2f}%'.format)

district_summary_df

,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,"39,170","$24,649,428",78.99,81.88,74.98%,85.81%,65.17%


In [34]:
## CSUMMARY ##

#grouping by schools to get the average scores
schools_grouped = school_data_complete.groupby('school_name').mean()
schools_start = schools_grouped.filter(['size','budget','reading_score','math_score'])

#creating the per-student column
schools_start['Per Student Budget'] = schools_start['budget']/schools_start['size']

#passing math, reading, and both
#grouping the data then getting the counts
math_passing_school = math_passing.groupby('school_name').count()['math_score']
reading_passing_school = reading_passing.groupby('school_name').count()['reading_score']
both_passing_school = both_passing.groupby('school_name').count()['student_name']
#creating the column
schools_start['% Passing Math'] = round((math_passing_school/schools_start['size']*100),2)
schools_start['% Passing Reading'] = round((reading_passing_school/schools_start['size']*100),2)
schools_start['% Passing Overall'] = round((both_passing_school/schools_start['size']*100),2)

#getting the scool type column
sorted_schools = school_data.sort_values(by = ['school_name'])
schools_index = sorted_schools.set_index('school_name')
schools_start['School Type'] = schools_index['type']

#formatting the column names
schools_renamed = schools_start.rename(columns = {'size': 'Total Students', 'budget': 'Total School Budget',
                                                 'reading_score': 'Average Reading Score', 
                                                  'math_score': 'Average Math Score', })
schools_reorganized = schools_renamed[['School Type', 'Total Students', 'Total School Budget','Per Student Budget',
                                       'Average Math Score', 'Average Reading Score', '% Passing Math',
                                       '% Passing Reading', '% Passing Overall']]
schools_reorganized.index.name = None

#formatting the columns
#making the change to integers
schools_reorganized['Total Students'] = schools_reorganized['Total Students'].astype('int')
#final formatting
schools_reorganized['Total Students'] = schools_reorganized['Total Students'].map('{:,}'.format)
schools_reorganized['Total School Budget'] = schools_reorganized['Total School Budget'].map('${:,.2f}'.format)
schools_reorganized['Per Student Budget'] = schools_reorganized['Per Student Budget'].map('${:,.2f}'.format)
schools_reorganized['Average Math Score'] = schools_reorganized['Average Math Score'].map('{:.2f}'.format)
schools_reorganized['Average Reading Score'] = schools_reorganized['Average Reading Score'].map('{:.2f}'.format)
schools_reorganized['% Passing Math'] = schools_reorganized['% Passing Math'].map('{:.2f}%'.format)
schools_reorganized['% Passing Reading'] = schools_reorganized['% Passing Reading'].map('{:.2f}%'.format)
schools_reorganized['% Passing Overall'] = schools_reorganized['% Passing Overall'].map('{:.2f}%'.format)

#print the table head
schools_reorganized.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%


In [35]:
## BEST 5 SCHOOLS ##

#filter by overall passing(decending)
best_schools = schools_reorganized.sort_values(by = ['% Passing Overall'], ascending = False)
#return the top 5 performing schools
best_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


In [36]:
##WORST 5 SCHOOLS##

#filter by overall passing (assending)
worst_schools = schools_reorganized.sort_values(by = ['% Passing Overall'])
#return the lowest performing schools
worst_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


In [37]:
## MATH SCORES BY GRADE ##

#find all of the 9th, 10th, 11th and 12th graders
ninth_grade = school_data_complete[school_data_complete['grade'] == "9th"]
tenth_grade = school_data_complete[school_data_complete['grade'] == "10th"]
eleventh_grade = school_data_complete[school_data_complete['grade'] == "11th"]
twelfth_grade = school_data_complete[school_data_complete['grade'] == "12th"]


#find the average math score for each grade
ninth_math = ninth_grade.groupby('school_name').mean()['math_score']
tenth_math = tenth_grade.groupby('school_name').mean()['math_score']
eleventh_math = eleventh_grade.groupby('school_name').mean()['math_score']
twelfth_math = twelfth_grade.groupby('school_name').mean()['math_score']

#create a data frame holding everything
math_scores = pd.DataFrame({'9th':ninth_math, '10th':tenth_math, '11th': eleventh_math, '12th': twelfth_math})
math_scores.index.name = None

#formatting the figures
math_scores['9th'] = math_scores['9th'].map('{:.2f}'.format)
math_scores['10th'] = math_scores['10th'].map('{:.2f}'.format)
math_scores['11th'] = math_scores['11th'].map('{:.2f}'.format)
math_scores['12th'] = math_scores['12th'].map('{:.2f}'.format)

#printing the table
math_scores

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


In [38]:
## READING SCORES BY GRADE ##

#find the average reading score for each grade
ninth_reading = ninth_grade.groupby('school_name').mean()['reading_score']
tenth_reading = tenth_grade.groupby('school_name').mean()['reading_score']
eleventh_reading = eleventh_grade.groupby('school_name').mean()['reading_score']
twelfth_reading = twelfth_grade.groupby('school_name').mean()['reading_score']

#creating a data frame for the individial grades
reading_scores = pd.DataFrame({'9th': ninth_reading, '10th': tenth_reading, 
                               '11th': eleventh_reading, '12th': twelfth_reading})
reading_scores.index.name = None

#formatting the figures
reading_scores['9th'] = reading_scores['9th'].map('{:.2f}'.format)
reading_scores['10th'] = reading_scores['10th'].map('{:.2f}'.format)
reading_scores['11th'] = reading_scores['11th'].map('{:.2f}'.format)
reading_scores['12th'] = reading_scores['12th'].map('{:.2f}'.format)

#printing the table
reading_scores

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


In [39]:
## SCORES BY SCHOOL SPENDING ##

#creating bins for the spending ranges (<584, 585-629, 630-644, 645-675)
spending_bins = [0, 584, 629, 644, 675]
#creating the labels
spending_labels = ['< $584', '$584-629', '$630-644', '$650-675']
#slicing the data and putting it into bins
schools_renamed['Spending Ranges (Per Student)'] = pd.cut(schools_renamed['Per Student Budget'], 
                                                              bins = spending_bins, labels = spending_labels)
#grouping by the new column
school_spending = schools_renamed.groupby('Spending Ranges (Per Student)').mean()
school_spending = school_spending[['Average Math Score', 'Average Reading Score', 
                                   '% Passing Math', '% Passing Reading', '% Passing Overall']]

#formatting the figures
school_spending['Average Math Score'] = school_spending['Average Math Score'].map('{:.2f}'.format)
school_spending['Average Reading Score'] = school_spending['Average Reading Score'].map('{:.2f}'.format)
school_spending['% Passing Math'] = school_spending['% Passing Math'].map('{:.2f}%'.format)
school_spending['% Passing Reading'] = school_spending['% Passing Reading'].map('{:.2f}%'.format)
school_spending['% Passing Overall'] = school_spending['% Passing Overall'].map('{:.2f}%'.format)

#printing the table
school_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Spending Ranges (Per Student),,,,,
< $584,83.46,83.93,93.46%,96.61%,90.37%
$584-629,81.90,83.16,87.13%,92.72%,81.42%
$630-644,78.52,81.62,73.48%,84.39%,62.86%
$650-675,77.00,81.03,66.16%,81.13%,53.53%


In [40]:
## SCORES BY SCHOOL SIZE ##

#create bins for a small school(>1000), medium school, (1000-2000), large school (2000+)
size_bins = [0, 1000, 2000, 5000]
#create the labels
size_labels = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000+)']
#slicing the data and putting it into bins
schools_renamed['School Size'] = pd.cut(schools_renamed['Total Students'], 
                                       bins = size_bins, labels = size_labels)
#groupig by the new column
school_size = schools_renamed.groupby('School Size').mean()
school_size = school_size[['Average Math Score', 'Average Reading Score', 
                                   '% Passing Math', '% Passing Reading', '% Passing Overall']]

#formatting the figures
school_size['Average Math Score'] = school_size['Average Math Score'].map('{:.2f}'.format)
school_size['Average Reading Score'] = school_size['Average Reading Score'].map('{:.2f}'.format)
school_size['% Passing Math'] = school_size['% Passing Math'].map('{:.2f}%'.format)
school_size['% Passing Reading'] = school_size['% Passing Reading'].map('{:.2f}%'.format)
school_size['% Passing Overall'] = school_size['% Passing Overall'].map('{:.2f}%'.format)

#printing the table
school_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,89.89%
Medium (1000-2000),83.37,83.86,93.60%,96.79%,90.62%
Large (2000+),77.75,81.34,69.96%,82.77%,58.29%


In [41]:
## SCORES BY SCHOOL TY ##

#change to index by school
school_type = schools_renamed.groupby('School Type').mean()
school_type = school_type[['Average Math Score', 'Average Reading Score', 
                                   '% Passing Math', '% Passing Reading', '% Passing Overall']]

#formatting the figures
school_type['Average Math Score'] = school_type['Average Math Score'].map('{:.2f}'.format)
school_type['Average Reading Score'] = school_type['Average Reading Score'].map('{:.2f}'.format)
school_type['% Passing Math'] = school_type['% Passing Math'].map('{:.2f}%'.format)
school_type['% Passing Reading'] = school_type['% Passing Reading'].map('{:.2f}%'.format)
school_type['% Passing Overall'] = school_type['% Passing Overall'].map('{:.2f}%'.format)

#printing the table
school_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%
